# LASSO Regression with Coordinate Descent Optimization

Here we solve regression problem based on a synthetic dataset. Then we use an optimized version of **coordinate descent** optimization algorithm to solve LASSO regression problem. At first we will talk about how have we generated the synthetic data set, followed by crude and optimized versions of LASSO solver using coordinate descent.  

The LASSO is the problem of solving,
$$\underset{\boldsymbol{w}, b}{arg \, min}\sum_i (\boldsymbol{w}^T\boldsymbol{x}_i + b - y_i)^2 + \lambda \sum_j |w_j|$$

Here $\boldsymbol{X}$ is a $d \times n$ matrix of data, and $\boldsymbol{x}_i$ is the $i^{th}$ column of the matrix. $\boldsymbol{y}$ is an $n \times 1$ vector of response variables, $\boldsymbol{w}$ is a $d$ dimensional weight vector, $b$ is a scalar offset term and $\lambda$ is a regularization tuning parameter.

A benefit of the LASSO is that of we believe many features are irrelevant for predicting $\boldsymbol{y}$, the LASSO can be used to enforce a sparse solution, effectively differentiating between the relevant and irrelevant features.

## Generating synthetic data
Suppose that there are $k < d$ number of features that are non-zero and features $k+1$ through $d$ are unnecessary (and potentially even harmful) for predicting $\boldsymbol{y}$.

Letting $N=250$, $d = 80$, $k=10$ and $\sigma = 1$ we generate data by drawing each element of $\boldsymbol{X} \in \mathbb{R}^{d \times N}$ from a standard normal distribution $\mathcal{N}(0,1)$.

Also we consider $b^*=0$ and create a $\boldsymbol{w}^*$ by setting the first $k=10$ elements to $\pm 10$ and the remaining elements to $0$.

Finally generate a Gaussian noise vector $\epsilon$ with variance $\sigma^2$ and form
$$\boldsymbol{y} = \boldsymbol{X}^T\boldsymbol{w}^* + b^* + \epsilon$$

With this synthetic data we solve multiple LASSO problems on a regularization path, starting at $\lambda_{max}$ and decreasing $\lambda$ by a constant ration of $2$ for $10$ steps.

To compute $\lambda_{max}$, the smallest value of $\lambda$ for which the solution $\boldsymbol{\hat{w}}$ is entirely zero is given by
$$\lambda_{max} = 2 \|\boldsymbol{X}\left(\boldsymbol{y} - \frac{1}{n}\sum_i y_i\right)\|_{\infty}$$
The above equation helps to choose the first $\lambda$ in a regularization path.

## Coordinate Descent for LASSO
The brute force way to implement coordinate descent is shown below

In [8]:
%%html
<img src="img/lasso1.png",width=15,height=15>

However we can reduce time complexity by avoiding the repeated expensive computations. Below I have presented an efficient version of coordinate descent to solve LASSO which took $<1 ms$ in MATLAB format

In [ ]:
while old_obj - new_obj > tol
        old_obj = new_obj;
        b = sum(data.y - data.X'*w_vec)/n;
        for k = 1:size(data.X, 1)
            ck = 2*data.X(k,:)*(data.y - data.X'*w_vec - b) + w_vec(k,1)*ak(1,k);
            if ck < -lam
                w_vec(k, 1) = (ck + lam)/ak(1, k);                
            elseif ck > lam
                w_vec(k, 1) = (ck - lam)/ak(1, k);                
            else
                w_vec(k, 1) = 0;                
            end
        end
        new_obj = current_obj(data.X, data.y, b, w_vec, lam);
        del_obj = old_obj - new_obj;
        
        % Stop before diverging
        if del_obj < 0
            break;
        end
end

## Checking the correctness of the answer
To ensure that a solution $(\boldsymbol{\hat{w}}, \hat{b})$ is correct, we also need to compute the value
$$2\boldsymbol{X}(\boldsymbol{X}^T\boldsymbol{\hat{w}} + \hat{b} - \boldsymbol{y})$$
The above will resul in a $d$ dimensional vector that should take the value $-\lambda sign(w_j)$ at $j$ for each $w_j$ that is non-zero. For the zero indices of $\boldsymbol{\hat{w}}$, this vector should take values lesser in magnitude than $\lambda$.

## Effect of meta-parameter $\lambda$ on non-zero features

In [10]:
%%html
<img src="img/non_zero_weights.png",width=15,height=15>

It can be seen from the figure that for very high values of $\lambda$, all of the weights estimated to be zero, however with deceased values of $\lambda$ the algorithm correctly predicted non-zero weights in $\boldsymbol{\hat{w}}$. Also when $\lambda$ value went down to very small, number of nonzero weights become more and more giving wrong predictions on non-zero weights.